In [ ]:
print("Hello World!")

In [2]:
import snowflake.connector
import os
import pandas as pd
import dotenv as dot
from sqlalchemy import create_engine
# from dotenv import load_dotenv

DATABASE_SCHEMA = "EVENT.DATATHON_2025_TEAM_ETA"

def get_snowflake_connection():
    """
    Create a connection to Snowflake using credentials from .env file
    """
    # Load environment variables
    dot.load_dotenv()
    
    # Get connection parameters from environment variables
    conn = snowflake.connector.connect(
        account=os.getenv("SNOWFLAKE_ACCOUNT"),
        user=os.getenv("SNOWFLAKE_USER"),
        password=os.getenv("SNOWFLAKE_PASSWORD"),
        role=os.getenv("SNOWFLAKE_ROLE"),
        warehouse=os.getenv("SNOWFLAKE_WAREHOUSE"),
        database=os.getenv("SNOWFLAKE_DATABASE"),
        schema=os.getenv("SNOWFLAKE_SCHEMA")
    )
    
    return conn

def query_to_df(query):
    """
    Execute a query and return the results as a pandas DataFrame
    """
    conn = get_snowflake_connection()
    cursor = conn.cursor()
    cursor.execute(query)

    # Get column names
    columns = [col[0] for col in cursor.description]

    # Fetch all rows and convert to list of dictionaries
    results = [dict(zip(columns, row)) for row in cursor.fetchall()]

    cursor.close()
    conn.close()

    return pd.DataFrame(results)

def upload_csv_to_snowflake(dataframe, table_name):
    
    conn = get_snowflake_connection()
    cursor = conn.cursor()
    
    engine = create_engine(
        f'snowflake://{conn.user}:{os.getenv("SNOWFLAKE_PASSWORD")}@{conn.account}/{conn.database}/{conn.schema}?warehouse={conn.warehouse}'
    )
    dataframe.to_sql(
        name=table_name,
        con=engine,
        schema=conn.schema,
        if_exists="replace",
        index=False,
        method='multi'
    )
    
    # Get row count
    cursor = conn.cursor()
    cursor.execute(f"SELECT COUNT(*) FROM {conn.database}.{conn.schema}.{table_name}")
    row_count = cursor.fetchone()[0]
    cursor.close()
    conn.close()
    
    return row_count
    

# Example status_df 
status = query_to_df(f"SELECT * FROM {DATABASE_SCHEMA}.status")
print(status.head())

   STATUSID        STATUS
0         1      Finished
1         2  Disqualified
2         3      Accident
3         4     Collision
4         5        Engine


TASK A

In [ ]:
# Create a master dataset from drivers, races, constructors, status, results

drivers = query_to_df(f"SELECT * FROM {DATABASE_SCHEMA}.DRIVERS")

races = query_to_df(f"SELECT * FROM {DATABASE_SCHEMA}.RACES")
races = races.rename(columns={'NAME': 'RACE_NAME'})

constructors = query_to_df(f"SELECT * FROM {DATABASE_SCHEMA}.CONSTRUCTORS")
constructors = constructors.rename(columns={'NAME': 'CONSTRUCTOR_NAME'})
constructors = constructors.rename(columns={'NATIONALITY': 'CONSTRUCTOR_NATIONALITY'})

results = query_to_df(f"SELECT * FROM {DATABASE_SCHEMA}.RESULTS")


main_df = results.merge(races, on="RACEID", suffixes=('', '_race')).merge(
    drivers, 
    on="DRIVERID", 
    suffixes=('', '_driver')
).merge(
    constructors, 
    on="CONSTRUCTORID", 
    suffixes=('', '_constructor')
).merge(
    status, 
    on="STATUSID", 
    suffixes=('', '_status')
)

print(main_df.head())

# Upload to Snowflake
row_count = upload_csv_to_snowflake(main_df, "CONSTRUCTOR_DRIVER_RACE_STATUS")

   RESULTID  RACEID  DRIVERID  CONSTRUCTORID  NUMBER  GRID  POSITION  \
0         1      18         1              1    22.0     1       1.0   
1         2      18         2              2     3.0     5       2.0   
2         3      18         3              3     7.0     7       3.0   
3         4      18         4              4     5.0    11       4.0   
4         5      18         5              1    23.0     3       5.0   

  POSITIONTEXT  POSITIONORDER POINTS  ...  QUALI_TIME SPRINT_DATE  \
0            1              1   10.0  ...        None        None   
1            2              2    8.0  ...        None        None   
2            3              3    6.0  ...        None        None   
3            4              4    5.0  ...        None        None   
4            5              5    4.0  ...        None        None   

   SPRINT_TIME          FULL_NAME         DOB NATIONALITY WINS  \
0         None     Lewis Hamilton  1985-01-07     British  105   
1         None      

In [ ]:
# Now we've created the big dataset, let's drop columns we don't need

columns_to_drop = ['POSITION', 'POSITIONTEXT','TIME', 'NUMBER']
columns_to_rename = [{'old': 'NATIONALITY', 'new': 'DRIVER_NATIONALITY'}, {'old':'DOB', 'new': 'DRIVER_DOB'}]

# Drop columns from the DataFrame
main_df_cleaned = main_df.drop(columns=columns_to_drop, errors='ignore')
print(f"Dropped {len(columns_to_drop)} columns. New shape: {main_df_cleaned.shape}")


conn = get_snowflake_connection()
cursor = conn.cursor()

# Drop columns from the existing table
for column in columns_to_drop:
    try:
        cursor.execute(f"ALTER TABLE {DATABASE_SCHEMA}.CONSTRUCTOR_DRIVER_RACE_STATUS DROP COLUMN IF EXISTS {column}")
        print(f"Dropped column {column} from table")
    except:
        print(f"Column {column} not found or could not be dropped")

# Rename columns from the existing table for clarity
for column in columns_to_rename:
    try:
        cursor.execute(f"ALTER TABLE {DATABASE_SCHEMA}.CONSTRUCTOR_DRIVER_RACE_STATUS RENAME COLUMN {column['old']} TO {column['new']}")
        print(f"Renamed column {column['old']} to {column['new']} in table")
    except:
        print(f"Column {column['old']} not found or could not be renamed")
cursor.close()
conn.close()

# Query the updated table to refresh the DataFrame
main_df_updated = query_to_df(f"SELECT * FROM {DATABASE_SCHEMA}.CONSTRUCTOR_DRIVER_RACE_STATUS")
print(f"Updated DataFrame shape: {main_df_updated.shape}")
print(main_df_updated.head())

Dropped 4 columns. New shape: (26759, 37)
Dropped column POSITION from table
Dropped column POSITIONTEXT from table
Dropped column TIME from table
Dropped column NUMBER from table
Renamed column NATIONALITY to DRIVER_NATIONALITY in table
Renamed column DOB to DRIVER_DOB in table
Updated DataFrame shape: (26759, 36)
   RESULTID  RACEID  DRIVERID  CONSTRUCTORID  GRID  POSITIONORDER POINTS  \
0         1      18         1              1     1              1   10.0   
1         2      18         2              2     5              2    8.0   
2         3      18         3              3     7              3    6.0   
3         4      18         4              4    11              4    5.0   
4         5      18         5              1     3              5    4.0   

   LAPS  MILLISECONDS  FASTESTLAP  ...  QUALI_DATE QUALI_TIME SPRINT_DATE  \
0    58     5690616.0        39.0  ...        None       None        None   
1    58     5696094.0        41.0  ...        None       None        Non

In [ ]:
# DNF column

main_df_updated = query_to_df(f"SELECT * FROM {DATABASE_SCHEMA}.CONSTRUCTOR_DRIVER_RACE_STATUS")

# Create DNF column based on multiple criteria
main_df_updated['DNF'] = (
    (main_df_updated['STATUSID'] != 1) |
    # Time-based criteria
    (main_df_updated['MILLISECONDS'].isnull()) |
    # Lap-based criteria (requires groupby to get max laps per race)
    main_df_updated.apply(lambda row: row['LAPS'] < 
                         main_df_updated[main_df_updated['RACEID'] == row['RACEID']]['LAPS'].max() * 0.9, 
                         axis=1)
)

# Print statistics
print(f"Total races: {len(main_df_updated)}")
print(f"DNF races: {main_df_updated['DNF'].sum()}")
print(f"Finished races: {len(main_df_updated) - main_df_updated['DNF'].sum()}")


Total races: 26759
DNF races: 19086
Finished races: 7673


In [ ]:
# Add DNF Col to Snowflake

# Update the table in Snowflake
conn = get_snowflake_connection()
cursor = conn.cursor()

# Update the table with the DNF values
cursor.execute(f"ALTER TABLE {DATABASE_SCHEMA}.CONSTRUCTOR_DRIVER_RACE_STATUS ADD COLUMN DNF BOOLEAN")
cursor.execute(f"""
    UPDATE {DATABASE_SCHEMA}.CONSTRUCTOR_DRIVER_RACE_STATUS 
    SET DNF = CASE 
        WHEN STATUSID = 1 AND MILLISECONDS IS NOT NULL THEN FALSE 
        ELSE TRUE 
    END
""")
cursor.close()
conn.close()

In [11]:
# Playing about with circuits

BOUNDARY_YEAR = 2014
circuits = query_to_df(f"SELECT * FROM {DATABASE_SCHEMA}.CIRCUITS")

winner_laps = results[results['POSITIONORDER'] == 1].groupby('RACEID')['LAPS'].max().reset_index()
winner_laps = winner_laps.merge(races[['RACEID', 'CIRCUITID', 'YEAR']], on='RACEID')
winner_laps = winner_laps[winner_laps['YEAR'] >= BOUNDARY_YEAR]

fastest_laps = results.dropna(subset=['FASTESTLAPTIME', 'FASTESTLAPSPEED'])
fastest_laps = fastest_laps.merge(races[['RACEID', 'CIRCUITID', 'YEAR']], on='RACEID')
fastest_laps = fastest_laps[fastest_laps['YEAR'] >= BOUNDARY_YEAR]


def lap_time_to_seconds(time):
    if(pd.isna(time)):
        return None
    try:
        parts = time.split(':')
        return float(parts[0]) * 60 + float(parts[1])
    except:
        return None

# Apply the conversion
fastest_laps['lap_time_seconds'] = fastest_laps['FASTESTLAPTIME'].apply(lap_time_to_seconds)
fastest_laps['FASTESTLAPSPEED'] = fastest_laps['FASTESTLAPSPEED'].astype(float)
fastest_laps['circuit_length_km'] = (fastest_laps['FASTESTLAPSPEED'] * fastest_laps['lap_time_seconds'] / 3600)

circuit_lengths = fastest_laps.groupby('CIRCUITID')['circuit_length_km'].mean().reset_index()
circuit_lengths = circuit_lengths.rename(columns={'circuit_length_km': 'ESTIMATED_LENGTH_KM'})
circuit_laps = winner_laps.groupby('CIRCUITID')['LAPS'].mean().reset_index()
circuit_laps = circuit_laps.rename(columns={'LAPS': 'AVG_RACE_LAPS'})

# Merge circuit information
enhanced_circuits = circuits.merge(circuit_lengths, on='CIRCUITID', how='left')
enhanced_circuits = enhanced_circuits.merge(circuit_laps, on='CIRCUITID', how='left')

# Calculate estimated race distance
enhanced_circuits['ESTIMATED_RACE_DISTANCE_KM'] = enhanced_circuits['ESTIMATED_LENGTH_KM'] * enhanced_circuits['AVG_RACE_LAPS']

# Display the enhanced circuits data
print(enhanced_circuits[['CIRCUITID', 'NAME', 'ESTIMATED_LENGTH_KM', 'AVG_RACE_LAPS', 'ESTIMATED_RACE_DISTANCE_KM']].head())


conn = get_snowflake_connection()
cursor = conn.cursor()

# Add new columns to the circuits table if they don't exist
cursor.execute(f"ALTER TABLE {DATABASE_SCHEMA}.CIRCUITS ADD COLUMN IF NOT EXISTS ESTIMATED_LENGTH_KM FLOAT")
cursor.execute(f"ALTER TABLE {DATABASE_SCHEMA}.CIRCUITS ADD COLUMN IF NOT EXISTS AVG_RACE_LAPS FLOAT")
cursor.execute(f"ALTER TABLE {DATABASE_SCHEMA}.CIRCUITS ADD COLUMN IF NOT EXISTS ESTIMATED_RACE_DISTANCE_KM FLOAT")

# Update the circuits table with the calculated values
for _, row in enhanced_circuits.iterrows():
    if pd.notna(row['ESTIMATED_LENGTH_KM']):
        cursor.execute(f"""
            UPDATE {DATABASE_SCHEMA}.CIRCUITS
            SET 
                ESTIMATED_LENGTH_KM = {row['ESTIMATED_LENGTH_KM']},
                AVG_RACE_LAPS = {row['AVG_RACE_LAPS'] if pd.notna(row['AVG_RACE_LAPS']) else 'NULL'},
                ESTIMATED_RACE_DISTANCE_KM = {row['ESTIMATED_RACE_DISTANCE_KM'] if pd.notna(row['ESTIMATED_RACE_DISTANCE_KM']) else 'NULL'}
            WHERE CIRCUITID = {row['CIRCUITID']}
        """)

cursor.close()
conn.close()


   CIRCUITID                            NAME  ESTIMATED_LENGTH_KM  \
0          1  Albert Park Grand Prix Circuit             5.294655   
1          2    Sepang International Circuit             5.543338   
2          3   Bahrain International Circuit             5.267601   
3          4  Circuit de Barcelona-Catalunya             4.659043   
4          5                   Istanbul Park             5.337986   

   AVG_RACE_LAPS  ESTIMATED_RACE_DISTANCE_KM  
0      57.666667                  305.325103  
1      56.000000                  310.426914  
2      59.416667                  312.983291  
3      66.000000                  307.496857  
4      58.000000                  309.603212  
